In [19]:
from collections import defaultdict
from pathlib import Path

In [20]:
# define file inputs
snpbed_path = Path("snpinfo.bed")
transcript_gtf_path = Path("hg38.p13.NCBI_RefSeq.gtf")
sg_intersects_path = Path("sg-intersects.txt")

# define file outputs
cisreg_genesets_out_path = Path("cisregulatory-genesets.txt")
chrom_genesets_out_path = Path("chrom-genesets.txt")

In [21]:
# run bedtools window to get genes within 1MB of each SNP
!bedtools window -w 1000000 -a {snpbed_path} -b {transcript_gtf_path} > {sg_intersects_path}

In [25]:
# get genes of interest in cis-regulatory regions
genes_cisreg = defaultdict(list)
with open(sg_intersects_path, 'r') as fobj:
    for line in fobj:
        # extract the relation between SNP and gene
        fields = line.strip().split('\t')
        rsid = fields[3]
        gene_id = fields[-1].split(';')[0].split(' ')[1]
        
        # place genes in the cis-regulatory vacinity into a dictionary for output
        genes_cisreg[rsid].append(gene_id)

# remove duplicates in each SNP gene set
cisreg_genesets_out = {}
for snp, genes in genes_cisreg.items():
    nonredundant = sorted(list(set(genes)))
    cisreg_genesets_out[snp] = nonredundant

# write the file out containing the cisregulatory gene sets
with open(cisreg_genesets_out_path,'w') as outobj:
    for snp, geneset in cisreg_genesets_out.items():
        genelist = ",".join(geneset)
        outobj.write(f"{snp}\t{genelist}\n")


In [26]:
# remove the temporary bedtools file
!rm {sg_intersects_path}

rm: sg-intersects.txt: No such file or directory


In [27]:
# get the chromosomes of our SNPs from the bed file
snp_chroms = {}
with open(snpbed_path, 'r') as fobj:
    for line in fobj:
        fields = line.strip().split('\t')
        chrom = fields[0]
        rsid = fields[3]
        snp_chroms[rsid] = chrom


In [44]:
# get gene_ids from the NCBI gtf corresponding to our SNP chroms
chrom_genes = defaultdict(list)
chroms_to_check = list(set([chrom for rsid, chrom in snp_chroms.items()]))
with open(transcript_gtf_path, 'r') as fobj:
    for line in fobj:
        fields = line.strip().split('\t')
        chrom = fields[0]
        if chrom in chroms_to_check:
            gene_id = fields[-1].split(';')[0].split(' ')[1]
            chrom_genes[chrom].append(gene_id)
        else:
            continue

# replace chroms with SNPs in final dict
chrom_genesets_out = {}
for rsid, chrom in snp_chroms.items():
    chrom_genesets_out[rsid] = list(set(chrom_genes[chrom]))

# write the file out
with open(chrom_genesets_out_path, 'w') as outobj:
    for snp, geneset in chrom_genesets_out.items():
        genelist = ",".join(geneset)
        outobj.write(f"{snp}\t{genelist}\n")